# Introduction to Atmospheric Correction
### By Susan Meerdink<sup>1</sup>

<sup>1</sup>The Machine Learning and Sensing Lab, Electrical and Computer Engineering, University of Florida, Gainesville, FL 32611
<url>https://faculty.eng.ufl.edu/machine-learning/</url>

**Electromagnetic Spectrum Refresher**

* Term used to describe the entire range of light that exists. Generally for remote sensing imagery we work with the visible shortwave infrared (VSWIR) which ranges from 350 to 2500 nm. However, it can also apply to the thermal infrared (TIR) which ranges from 2500 nm to 15 um. Atmospheric correction can be applied to imagery measuring the VSWIR and TIR, but this introduction will focus on the VSWIR.
<img src="Picture01.png" alt="Electromagnetic Spectrum" style="width: 600px;"/>

**What is atmospheric correction?**